In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [2]:
spark = SparkSession.builder.getOrCreate()

In [ ]:
df_access = spark.read.csv(path="access_logs_csv.csv", sep=";", header=True)
df_browser_site = spark.read.csv(path="official_browser_site_csv.csv", sep=";", header=True)
df_dict_site = spark.read.csv(path="dict_site_status_code_csv.csv", sep=";", header=True)

df_access.show()
df_browser_site.show()
df_dict_site.show()

In [ ]:
df1 = df_access.join(df_dict_site, df_access.site_status_code == df_dict_site.site_status_code, "inner")
df1 = df1.join(df_browser_site, df_browser_site.browser in df1.browser_raw.lower(), "inner")

df1.show(n=10)

In [ ]:
print("Rows cound:", df1.count())
df2 = df1.dropna()
print("Rows with na dropped:", df1.count() - df2.count())
print("Rows in result:", df2.count())

In [ ]:
addresses = df2.groupBy("site", "ip_address").count()
filtered_addresses = addresses.filter(addresses.count >= 5).select(addresses.site, addresses.count).distinct()
filtered_addresses.show()

In [ ]:
sorted_addresses = filtered_addresses.orderBy(filtered_addresses.count).desc().take(10)
sorted_addresses.show()

In [ ]:
from time import strptime

most_popular_between_time = df2.filter(6 <= strptime("%HH:%mm:%ss", df2.date_time).tm_hour < 11).select(df2.site, df2.date_time)
most_popular_between_time = most_popular_between_time.groupBy(most_popular_between_time.site).count()
most_popular_between_time.orderBy(most_popular_between_time.count).desc().take(10)
most_popular_between_time.show()